# CoKrigeage RMQS - Mousse

Dans ce notebook, on cherche à effectuer le krigeage de la concentration dans l'air en un certain élément à partir du [jeu de données RMQS](../data/RMQS.csv). Pour améliorer cet estimation de la varibale, on cherche ensuite à effectuer une co-krigeage à l'aide des [données de concentration de cet élément dans les mousses](../data/moss.csv).

:warning: Il est toutefois important de noter que les données RMQS ne remontent que jusqu'à 2009 tandis que les données sur les mousses datent de 2021. cet écart signicatif pourrait donc être la cause d'éventuels résultats médiocres. :warning:

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bramm_data_analysis import loaders
from pathlib import Path
import gstlearn as gl
import gstlearn.plot as gp
from bramm_data_analysis.matching import Matcher
from bramm_data_analysis.loaders.preprocessing import QuantileThreshold
import matplotlib.pyplot as plt
import numpy as np

## Chargement des Données

### Définition des variables d'intérêt

In [ ]:
z_moss = "aluminium"
z_rmqs = "al_tot_hf"

### Chargement des Données

La gestion des valeurs dupliquées se fait en deux étapes. Dans un premier temps, on aggrège les données correspondants à un même échantillon (les données ayant même date, même longitude et même latitude), en faisant leur moyenne. Ensuite, si il reste des données ayant la même position spatial (il n'est plus question de date ici), on conserve les données les plus récentes.

On s'assure ensuite de l'absence de `NaN` dans les données et on convertit le jeu de données (on conservant seulement la longitude, la latitute et les valeurs de cuivre) au format `Db` de `gstlearn` pour effectuer un premier krigeage.

Se référer au module [loaders](../src/bramm_data_analysis/loaders/__init__.py) pour plus d'informations.

##### DataFrame Mousse

In [ ]:
moss_data_path = Path("../data/Mines_2024.xlsx")
df_moss = loaders.from_moss_csv(moss_data_path).retrieve_filtered_df(
    fields=["longitude", "latitude", "date", z_moss],
    duplicates_handling_strategy="mean",
    thresholds=[QuantileThreshold(field=z_moss, lower=0.05, upper=0.95)],
)
# A threshold is set to remove potential outliers.
# It consists in the removal of the top and bottom 5% of values of the data.

##### DataFrame RMQS

In [ ]:
rmqs_data_path = Path("../data/RMQS.csv")
df_rmqs = loaders.from_rmqs_csv(rmqs_data_path).retrieve_filtered_df(
    fields=["longitude", "latitude", "date_complete", z_rmqs],
    duplicates_handling_strategy="mean",
    thresholds=[QuantileThreshold(field=z_rmqs, lower=0.05, upper=0.95)],
)
# A threshold is set to remove potential outliers.
# It consists in the removal of the top and bottom 5% of values of the data.

##### Correspondance Mousse RMQS

Les données de Mousse et RMQS n'étant pas échantillonnées aux mêmes endroits, il est nécessaire de faire correspondre les lieu d'échantillonnage de mousse et des données RMQS.

Pour cela, le module [matching](../src/bramm_data_analysis/matching.py) définit un ensemble d'objects permettant, sur la base de l'[algorithme des plus proches voisins implémenté par sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html), de récupérer les données RMQS situées à proximité des sites d'échantillonnage des mousses. On définit pour cela un rayon maximal d'action, `km_threshold`. S'il n'y a pas de données RMQS dans la sphère définie par ce rayon, le site d'échantillonnage de mousse est écarté pour la suite. 

Une fois cette correspondance effectuée, on conserve les données de mousses auxquelles ont été rapprochées des données RMQS en tant que "jeu d'entraînement" et les données RMQS non attribuées (appelées dans la suite `leftovers`) en tant que "jeu de validation".

In [ ]:
# Matching
data_matcher = Matcher(km_threshold=10, year_threshold=2000)
matched_df, leftovers = data_matcher.match_rmqs_to_moss(
    df_moss,  # Moss Data
    df_rmqs,  # RMQS Data
    radians=False,  # Longitude and Latitude are in degree
    leftovers=True,  # Output Leftovers
)
print(
    f" Year Threshold : {data_matcher.year_threshold} \n",
    f"Distance Threshold : {data_matcher.km_threshold} km \n",
    f"Conserved : {matched_df.shape[0]} / {df_moss.shape[0]}",
)
# Visualisation
# Plot Leftovers
plt.scatter(
    leftovers["longitude"],
    leftovers["latitude"],
    label="RMQS Leftovers",
    alpha=0.4,
    color="grey",
)
# Plot Moss' convserved data points.
plt.scatter(
    matched_df[f"longitude{data_matcher.moss_suffix}"],
    matched_df[f"latitude{data_matcher.moss_suffix}"],
    label="Conserved Moss Samples",
    alpha=0.5,
    color="green",
)
# Plot RMQS' conserved data points.
plt.scatter(
    matched_df[f"longitude{data_matcher.rmqs_suffix}"],
    matched_df[f"latitude{data_matcher.rmqs_suffix}"],
    label="Matched RMQS DataPoints",
    alpha=0.5,
    color="red",
)
plt.title("RMQS - Moss Matching Visualization")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.plot()

Une fois cette correspondance effectuée, on dispose donc d'un jeu de données constitué d'une grille de points possédant chacun une valeur de concentration de métal dans l'air (depuis les données RMQS) et une valeur de concentration de métal dans les mousses (depuis les données de mousses).

##### Conversion en `Db`

On convertit ensuite les `DataFrame` en `Db` afin d'utiliser la libraire [`gstlearn`](https://gstlearn.org/).

In [ ]:
# Rename longitude and latitude fields.
sliced_df = matched_df.rename(
    columns={"longitude_moss": "longitude", "latitude_moss": "latitude"}
).filter(["latitude", "longitude", z_moss, z_rmqs])

# Convert matched data to Db and set locators
db_match = gl.Db_fromPanda(sliced_df)
db_match.setLocators(["longitude", "latitude"], gl.ELoc.X)
db_match.setLocator(z_rmqs, gl.ELoc.Z)

# Convert unmatched RMQS data to Db and set locators
db_leftovers = gl.Db_fromPanda(leftovers)
db_leftovers.setLocators(["longitude", "latitude"], gl.ELoc.X)
db_leftovers.setLocator(z_rmqs, gl.ELoc.Z)

## Krigeage Ordinaire

### Variogramme

On effectue dans un premier temps un variogramme pour analyser les données. 

In [ ]:
varioParamMulti = gl.VarioParam.createMultiple(ndir=2, npas=8, dpas=0.5)
vario2dir = gl.Vario(varioParamMulti)
err = vario2dir.compute(db_match)
fitmod = gl.Model()
types = [gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.GAUSSIAN]
err = fitmod.fit(vario2dir, types=types)
ax = gp.varmod(vario2dir, fitmod)

### Krigeage

On effectue un krigeage ordinaire dans un premier temps, en utilisant uniquement les données RMQS du "jeu d'entraînement".

In [ ]:
err = gl.kriging(
    dbin=db_match,
    dbout=db_leftovers,
    model=fitmod,
    neigh=gl.NeighUnique.create(),  # Use Unique Neighborhood
    flag_est=True,
    flag_std=True,
    flag_varz=False,
    namconv=gl.NamingConvention("OK"),
)

##### Visualisation des Résultats du Krigeage

On va ensuite visualiser :
- Les données réelles
- Les données obtenues par krigeage
- L'écart de krigeage
- L'écart relatif $\frac{valeurs\_reelles - valeurs\_predites}{valeurs\_reelles}$

In [ ]:
vmin = np.min(db_leftovers[z_rmqs])
vmax = np.max(db_leftovers[z_rmqs])

fig, axes = plt.subplots(2, 2, figsize=(20, 20))

# Real Values
fig.add_subplot(2, 2, 1)
ax = db_match.plot(flagCst=True, color="red")
ax = db_leftovers.plot(
    f"{z_rmqs}",
    flagLegendColor=True,
    zorder=-1,
    vmin=vmin,
    vmax=vmax,
    size=50,
)
ax.decoration(title=f"{z_rmqs} - True Values")

# Predicted Values
fig.add_subplot(2, 2, 2)
ax = db_match.plot(flagCst=True, color="red")
ax = db_leftovers.plot(
    f"OK.{z_rmqs}.estim",
    flagLegendColor=True,
    zorder=-1,
    vmin=vmin,
    vmax=vmax,
    size=50,
)
ax.decoration(title=f"{z_rmqs} - Ordinary Kriging")

# Kriging Standard deviation
fig.add_subplot(2, 2, 3)
ax = db_match.plot(flagCst=True, color="red")
ax = db_leftovers.plot(
    f"OK.{z_rmqs}.stdev",
    flagLegendColor=True,
    zorder=-1,
    size=50,
)
ax.decoration(title=f"{z_rmqs} - Ordinary Kriging (stdev)")

# Relative Error
true_vals = db_leftovers[z_rmqs]
pred_vals = db_leftovers[f"OK.{z_rmqs}.estim"]
db_leftovers["absolute_error"] = (true_vals - pred_vals) / true_vals

fig.add_subplot(2, 2, 4)
db_match.plot(flagCst=True, color="red")
ax = db_leftovers.plot(
    "absolute_error",
    flagLegendColor=True,
    zorder=-1,
    size=50,
)
ax.decoration(title=f"{z_rmqs} - Relative Error")
plt.show()

## CoKrigeage

On effectue ensuite un Cokrigeage, en utilisant à la fois les données RMQS du "jeu d'entraînement" ainsi que les données des mousses.

In [ ]:
# Update Locators
db_match.setLocators([z_rmqs, z_moss], gl.ELoc.Z)

### Variogramme

In [ ]:
varioexp2var = gl.Vario.create(varioParamMulti)
err = varioexp2var.compute(db_match)
fitmod2var = gl.Model()
err = fitmod2var.fit(
    varioexp2var,
    types=[gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.CUBIC, gl.ECov.LINEAR],
)
fitmod2var.setDriftIRF(0, 0)
ax = gp.varmod(varioexp2var, fitmod2var, lw=2)
gp.decoration(ax, title=f"{z_rmqs} and {z_moss}")

### Krigeage

In [ ]:
err = gl.kriging(
    dbin=db_match,
    dbout=db_leftovers,
    model=fitmod2var,
    neigh=gl.NeighUnique.create(),  # Use Unique Neighborhood
    namconv=gl.NamingConvention.create(prefix="COK"),
)

##### Visualisation des Résultats du Krigeage

On va ensuite visualiser :
- Les données réelles
- Les données obtenues par krigeage
- L'écart de krigeage
- L'écart relatif $\frac{valeurs\_reelles - valeurs\_predites}{valeurs\_reelles}$

In [ ]:
vmin = np.min(db_leftovers[z_rmqs])
vmax = np.max(db_leftovers[z_rmqs])

fig, axes = plt.subplots(2, 2, figsize=(20, 20))

# Real Values
fig.add_subplot(2, 2, 1)
ax = db_match.plot(flagCst=True, color="red")
ax = db_leftovers.plot(
    f"{z_rmqs}",
    flagLegendColor=True,
    zorder=-1,
    vmin=vmin,
    vmax=vmax,
    size=50,
)
ax.decoration(title=f"{z_rmqs} - True Values")

# Predicted Values
fig.add_subplot(2, 2, 2)
ax = db_match.plot(flagCst=True, color="red")
ax = db_leftovers.plot(
    f"COK.{z_rmqs}.estim",
    flagLegendColor=True,
    zorder=-1,
    vmin=vmin,
    vmax=vmax,
    size=50,
)
ax.decoration(title=f"{z_rmqs} - CoKriging")

# Kriging Standard deviation
ax0 = fig.add_subplot(2, 2, 3)
ax = db_match.plot(flagCst=True, color="red")
ax = db_leftovers.plot(
    f"COK.{z_rmqs}.stdev",
    flagLegendColor=True,
    zorder=-1,
    size=50,
)
ax.decoration(title=f"{z_rmqs} - CoKriging (stdev)")

# Relative Error
true_vals = db_leftovers[z_rmqs]
pred_vals = db_leftovers[f"COK.{z_rmqs}.estim"]
db_leftovers["absolute_error"] = (true_vals - pred_vals) / true_vals

fig.add_subplot(2, 2, 4)
db_match.plot(flagCst=True, color="red")
ax = db_leftovers.plot(
    "absolute_error",
    flagLegendColor=True,
    zorder=-1,
    size=50,
)
ax.decoration(title=f"{z_rmqs} - Relative Error")
plt.show()

## Comparaison des Résultats

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
ordinary_kriging_score = np.sqrt(
    mean_squared_error(
        db_leftovers[z_rmqs], db_leftovers[f"COK.{z_rmqs}.estim"]
    )
)

cokriging_score = np.sqrt(
    mean_squared_error(
        db_leftovers[z_rmqs], db_leftovers[f"OK.{z_rmqs}.estim"]
    )
)
print(
    f"Ordinary Kriging Results : {ordinary_kriging_score}\n",
    f"CoKriging Results : {cokriging_score}\n",
    sep="",
)

Le Co-Krigeage semble améliorer faiblement les résultats, en terme de RMSE.

In [ ]:
opers = gl.EStatOption.fromKeys(["NUM", "MINI", "MAXI", "MEAN", "STDV"])
gl.dbStatisticsPrint(
    db_leftovers,
    names=([f"OK.{z_rmqs}.*"]),
    opers=opers,
    title="Statistics on the Ordinary Kriging predictions",
)
gl.dbStatisticsPrint(
    db_leftovers,
    names=([f"COK.{z_rmqs}.*"]),
    opers=opers,
    title="Statistics on the Ordinary CoKriging predictions",
)

On constate que le co-krigeage augmente légèrement l'étendue des données prédites et augmente également légèrement l'écart type des données prédites (dans le cas de l'aluminium).

Pour explorer plus en détail le jeu de données, n'héstez pas à rejouer ce notebook avec différentes variables, par exemple :

```python
z_moss = "iron"
r_rmqs = "fe_tot_hf"
```

```python
z_moss = "copper"
r_rmqs = "cu_tot_hf"
```

``` python
z_moss = "calcium"
r_rmqs = "ca_tot_hf"
```

Il est également possible de modifier le rayon lors de la correspondance RMQS - Mousse, pour augmenter (resp diminuer) le nombre de points dans le jeu de données en augmentant (resp diminuant) la valeur de `km_threshold`, dans la section [Correspondance des données Mousse et RMQS](#correspondance-mousse-rmqs).